## 当代人工智能 Project1

RoBERTa部分

In [ ]:
import numpy as np
import pandas as pd
import random

先在Colab中安装Transformers库

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 3.3 MB 25.4 MB/s 
     |████████████████████████████████| 636 kB 48.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import RobertaTokenizer, RobertaModel
from transformers import Trainer, TrainingArguments

In [ ]:
from transformers import RobertaForSequenceClassification

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [ ]:
import torch

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
roberta_model = "roberta-base"

Tokenizer仅初始化了一次，做不同的分类任务请重开一个notebook

In [ ]:
roberta_model_tokenizer = RobertaTokenizer.from_pretrained(roberta_model, do_lower_case=True)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

此函数实现从20newsgroup中读取所有数据，并且完成数据集分割

In [ ]:
def newsgroup_read(test_size=0.2):
  dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
  news_text = dataset.data
  news_label = dataset.target
  return train_test_split(news_text, news_label, test_size=test_size), dataset.target_names

次函数实现从20newsgroup中读取'alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space'，'comp.os.ms-windows.misc'五个主题的数据，并且完成数据集分割

In [ ]:
def newsgroup_read_selected(test_size=0.2):
  categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space', 'comp.os.ms-windows.misc']
  dataset = fetch_20newsgroups(subset='all', shuffle=True, remove=("headers", "footers", "quotes"), categories = categories)
  news_text = dataset.data
  news_label = dataset.target
  return train_test_split(news_text, news_label, test_size=test_size), dataset.target_names

In [ ]:
def newsgroup_read_four_cat(test_size=0.2):
  categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.med']
  dataset = fetch_20newsgroups(subset='all', shuffle=True, remove=("headers", "footers", "quotes"), categories = categories)
  news_text = dataset.data
  news_label = dataset.target
  return train_test_split(news_text, news_label, test_size=test_size), dataset.target_names

In [ ]:
(train_news_texts, valid_news_texts, train_news_labels, valid_news_labels), target_names = newsgroup_read()

In [ ]:
(selected_train_news_texts, selected_valid_news_texts, selected_train_news_labels, selected_valid_news_labels), selected_target_names = newsgroup_read_selected()

In [ ]:
(four_train_news_texts, four_valid_news_texts, four_train_news_labels, four_valid_news_labels), four_target_names = newsgroup_read_four_cat()

In [ ]:
max_length = 256
# max_length = 512
# 512在大部分情况下Colab会报出内存不足的错误

In [ ]:
roberta_train_encodings = roberta_model_tokenizer(selected_train_news_texts, truncation=True, padding=True, max_length=max_length)
roberta_valid_encodings = roberta_model_tokenizer(selected_valid_news_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
roberta_train_encodings_four_class = roberta_model_tokenizer(four_train_news_texts, truncation=True, padding=True, max_length=max_length)
roberta_valid_encodings_four_class = roberta_model_tokenizer(four_valid_news_texts, truncation=True, padding=True, max_length=max_length)

在这个RoBERTa实验中，我们使用五分类任务

In [ ]:
class GetNewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = GetNewsGroupsDataset(roberta_train_encodings_four_class, four_train_news_labels)
valid_dataset = GetNewsGroupsDataset(roberta_valid_encodings_four_class, four_valid_news_labels)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(roberta_model,num_labels=len(four_target_names)).to("cuda") # 加载roberta model，并且使用Google Colab 的 GPU训练

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

以下为5分类任务的训练

In [ ]:
training_args = TrainingArguments(
    output_dir='./roberta_model_results',         
    num_train_epochs=10,             
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=12,   
    warmup_steps=200,               
    weight_decay=0.01,             
    logging_dir='./logs',           
    load_best_model_at_end=True,    
    logging_steps=250,             
    evaluation_strategy="steps",    
)

In [ ]:
def compute_model_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  model_acc = accuracy_score(labels, preds)
  return {
      'roberta_model_accuracy': model_acc,
  }

trainer = Trainer(
    model=model,                      
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=valid_dataset,         
    compute_metrics=compute_model_metrics,    
)

开始Fine Tune RoBERTa模型

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3497
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4380


Step,Training Loss,Validation Loss,Roberta Model Accuracy
250,1.051600,0.748830,0.713143
500,0.733400,0.666042,0.713143
750,0.583900,0.598088,0.771429
1000,0.480000,0.729835,0.761143
1250,0.438300,0.657302,0.794286
1500,0.365000,0.896505,0.793143
1750,0.358900,0.882012,0.772571
2000,0.248500,1.008656,0.780571
2250,0.198000,1.073585,0.785143
2500,0.185000,1.060534,0.790857


***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./roberta_model_results/checkpoint-250
Configuration saved in ./roberta_model_results/checkpoint-250/config.json
Model weights saved in ./roberta_model_results/checkpoint-250/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./roberta_model_results/checkpoint-500
Configuration saved in ./roberta_model_results/checkpoint-500/config.json
Model weights saved in ./roberta_model_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./roberta_model_results/checkpoint-750
Configuration saved in ./roberta_model_results/checkpoint-750/config.json
Model weights saved in ./roberta_model_results/checkpoint-750/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 875
  Batch size = 12
Saving model checkpoint to ./roberta_model_results/checkpoin

TrainOutput(global_step=4380, training_loss=0.3051620516058517, metrics={'train_runtime': 3781.1123, 'train_samples_per_second': 9.249, 'train_steps_per_second': 1.158, 'total_flos': 4600620720983040.0, 'train_loss': 0.3051620516058517, 'epoch': 10.0})

In [ ]:
以下为四分类的训练

In [ ]:
training_args = TrainingArguments(
    output_dir='./roberta_four_class_model_results',       
    num_train_epochs=10,             
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=12, 
    warmup_steps=200,             
    weight_decay=0.01,           
    logging_dir='./logs',         
    load_best_model_at_end=True,     
    logging_steps=250,              
    evaluation_strategy="steps",   
)

In [ ]:
def compute_model_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  model_acc = accuracy_score(labels, preds)
  return {
      'roberta_model_accuracy': model_acc,
  }

trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,      
    eval_dataset=valid_dataset,         
    compute_metrics=compute_model_metrics,    
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 2712
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3390


Step,Training Loss,Validation Loss,Roberta Model Accuracy
250,0.842900,0.905619,0.679941
500,0.550600,0.647354,0.784661
750,0.448800,0.597781,0.814159
1000,0.364300,0.874650,0.792035
1250,0.328600,0.633260,0.824484
1500,0.251000,0.839118,0.815634
1750,0.193100,0.878693,0.836283
2000,0.125600,1.028521,0.814159
2250,0.118400,1.007266,0.828909
2500,0.072300,1.100358,0.830383


***** Running Evaluation *****
  Num examples = 678
  Batch size = 12
Saving model checkpoint to ./roberta_four_class_model_results/checkpoint-250
Configuration saved in ./roberta_four_class_model_results/checkpoint-250/config.json
Model weights saved in ./roberta_four_class_model_results/checkpoint-250/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 678
  Batch size = 12
Saving model checkpoint to ./roberta_four_class_model_results/checkpoint-500
Configuration saved in ./roberta_four_class_model_results/checkpoint-500/config.json
Model weights saved in ./roberta_four_class_model_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 678
  Batch size = 12
Saving model checkpoint to ./roberta_four_class_model_results/checkpoint-750
Configuration saved in ./roberta_four_class_model_results/checkpoint-750/config.json
Model weights saved in ./roberta_four_class_model_results/checkpoint-750/pytorch_model.bin
***** Running Evaluation *****
